
# Matthew Muller
1/19/23

Description:
- This script will train and evaluate a machine learning model using sklearn.

### Under the hood

In [ ]:
########################################
# Library Imports
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
from scipy.stats import kruskal

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.svm import LinearSVC, SVC
from sklearn.decomposition import PCA, NMF
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.metrics import roc_curve, confusion_matrix

from joblib import load, dump

########################################
# Set/Append Working directory)

########################################
# Import Functions
from MattTools.plotting import plot_roc_curve, plot_confusion_matrix, plot_training_roc_curve_ci, plot_roc_curve_ci
from MattTools.plotting import plot_scree, plot_training_probas
from MattTools.stats import bootstrap_auc_confidence
from MattTools import utils

# Hide warnings
utils.hide_warnings()
utils.set_random_seed()

### Load in Data

In [ ]:
path = '../data/clean/'
X_train = pd.read_csv(path+'pace/features.csv', index_col=0, header=0)
y_train = pd.read_csv(path+'pace/labels.csv').to_numpy()[:,0]

X_test = pd.read_csv(path+'duke/features_group1.csv')
y_test = pd.read_csv(path+'duke/labels_group1.csv').to_numpy()[:,0]

X_test2 = pd.read_csv(path+'duke/features_group2.csv')
y_test2 = pd.read_csv(path+'duke/labels_group2.csv').to_numpy()[:,0]

X_test.shape, X_train.shape

### Run Model

In [ ]:
#### Make model pipeline (if needed) and search for params
# The general idea here is fit each gene to a SVC and then add them into a voting classifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

# good RF random seeds:
seeds = [1148093, 1095286, 1665788, 97057, 152878, 4543, 277452, 295106, 191278, 701043, 81388, 951209, 1327001, 527903, 1148093, 1095286, 1665788, 97057, 152878, 4543, 277452, 295106, 191278, 701043, 81388, 951209, 1327001, 527903]
n = 28
# seeds = [randint(0,1000000) for x in range(n)]
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    # make a voting classifier made a bunch of RFs
    ('voting', VotingClassifier(
        estimators=
        [('rf'+str(i), RandomForestClassifier(class_weight='balanced', n_estimators=100)) for i in range(n)] +
        
        [('extraTrees'+str(i), ExtraTreesClassifier(class_weight='balanced', random_state=seeds[i])) for i in range(n) ] +
        
        [('gbc'+str(i), GradientBoostingClassifier(random_state=seeds[i], max_features='log2', n_estimators=60)) for i in range(n) ] +
        
        [('ada'+str(i), AdaBoostClassifier(learning_rate=10)) for i in range(n) ],
        
        voting='soft',
        n_jobs=-1,
        ))
    ])

cv = GridSearchCV(
    pipe, parameters,
    n_jobs=-1,
    scoring="roc_auc",
    )

# Fit model
cv.fit(X_train, y_train)

model = cv.best_estimator_
print(cv.best_score_)
print(cv.best_params_)

### Evaludate Model

In [ ]:
from sklearn.model_selection import StratifiedKFold
# Training Stats

cv = StratifiedKFold(n_splits=10)

plot_training_roc_curve_ci(model, X_train, y_train, title='10-fold Cross Validation Training on PACE Cohort',cv = cv)
plot_training_probas(
    model, X_train, y_train, title='10-fold Cross Validation Training on PACE Cohort',cv = cv)

In [ ]:

#### Metrics
print(classification_report(y_test, model.predict(X_test)))
print(f'AUC: {roc_auc_score(y_test, model.predict_proba(X_test)[:,1])}')
plot_confusion_matrix(y_test, model.predict(X_test), labels = ['normal', 'hyper'])
plot_roc_curve_ci(model, X_test, y_test, title='ROC Curve on Duke Cohort')

# plot the prc curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import auc

# predict probabilities
lr_probs = model.predict_proba(X_test)[:,1]
# calculate precision and recall for each threshold
lr_precision, lr_recall, _ = precision_recall_curve(y_test, lr_probs)
# calculate scores
lr_auc = auc(lr_recall, lr_precision)
# summarize scores
print('Logistic: PR AUC=%.3f' % (lr_auc))
# plot the precision-recall curves
no_skill = len(y_test[y_test==1]) / len(y_test)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(lr_recall, lr_precision, marker='.', label='Logistic')
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
# show the plot
plt.show()

### Model Disease Correlations

In [ ]:
# ### Load in disease gene ratio data
# sle_ratios = pd.read_csv('../data/clean/model_validation/sle/sle_features_labels.csv', index_col=0)
male_ratios = pd.read_csv('../data/clean/model_validation/male/male_features_labels.csv', index_col=0)
mace_ratios = pd.read_csv('../data/clean/model_validation/mace/mace_features_labels.csv', index_col=0)
covid_ratios = pd.read_csv('../data/clean/model_validation/covid/covid_features_labels.csv', index_col=0)
pad_ratios = pd.read_csv('../data/clean/model_validation/pad/pad_features_labels.csv', index_col=0)
harp_ratios = pd.read_csv('../data/clean/model_validation/harp/harp_features_labels.csv', index_col=0)

disease_ratios = {
    # 'sle':sle_ratios, 
    'male':male_ratios, 
    'mace':mace_ratios, 
    'covid':covid_ratios, 
    'pad':pad_ratios, 
    'harp':harp_ratios
    }

male_ratios